In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%cd ..

In [ ]:
import wrappingnet.utils as utils
from wrappingnet.dataloaders import manifold40_dset
from wrappingnet.models import WrappingNet_sphere_LC, WrappingNet_global_basesup3

import os
import tqdm
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

device = 'cuda:1' if torch.cuda.is_available() else 'cpu'

In [ ]:
dset_train = manifold40_dset(root='./datasets/Manifold40', train=True)
dset_test = manifold40_dset(root='./datasets/Manifold40', train=False)
len(dset_train), len(dset_test)


In [ ]:
"""Loop 3 times"""
params = {'loss_func':'MSL2', 'dset':'manifold40', 'latent_dim':512, 'extra':'global_basesup3'}
model = WrappingNet_global_basesup3(input_dim=7, feature_dim=params['latent_dim'], num_loop=3)
saved = torch.load(f"trained/MeshAE_{params['loss_func']}_{params['dset']}_d{params['latent_dim']}{params['extra']}.ckpt", map_location='cpu')
model.load_state_dict(saved)
model.eval()

In [ ]:
"""Loop 3 times"""
params = {'loss_func':'MSL2', 'dset':'manifold40', 'latent_dim':512, 'extra':'LC'}
model = WrappingNet_sphere_LC(input_dim=7, feature_dim=params['latent_dim'], num_loop=3)
saved = torch.load(f"trained/MeshAE_{params['loss_func']}_{params['dset']}_d{params['latent_dim']}{params['extra']}.ckpt", map_location='cpu')
model.load_state_dict(saved)
model.eval()

In [ ]:
model = model.to(device)

z_train = []
for i, mesh in tqdm.tqdm(enumerate(dset_train), total=len(dset_train)):
    mesh = mesh.to(device)
    with torch.no_grad():
        mesh.pos = utils.normalize_pos(mesh.pos)
        face_base, features = model.encoder(mesh.pos, mesh.face.T)
        features = model.mlp(features)
        latent_code = torch.max(features, dim=0)[0].unsqueeze(0) 
        latent_code = model.mlp2(latent_code).squeeze() 
    z_train.append(latent_code)
z_train = torch.stack(z_train).cpu().numpy()

z_test = []
for i, mesh in tqdm.tqdm(enumerate(dset_test), total=len(dset_test)):
    mesh = mesh.to(device)
    with torch.no_grad():
        face_base, features = model.encoder(mesh.pos, mesh.face.T)
        features = model.mlp(features)
        latent_code = torch.max(features, dim=0)[0].unsqueeze(0) 
        latent_code = model.mlp2(latent_code).squeeze() 
    z_test.append(latent_code)
z_test = torch.stack(z_test).cpu().numpy()

z_total = np.concatenate((z_train, z_test), axis=0)

In [ ]:
labels_train = np.repeat(np.arange(30), 16)
labels_test = np.repeat(np.arange(30), 4)
labels_train = np.repeat(np.arange(30), 200)
labels_test = np.repeat(np.arange(30), 200)
labels_total = np.concatenate((labels_train, labels_test))

In [ ]:
import pandas as pd
info = pd.read_csv('./evaluation/metadata_modelnet40.csv')
test_idx = np.where(info['split']=='test')
test_labels  = info['class'].to_numpy()[test_idx]
train_idx = np.where(info['split']=='train')
train_labels = info['class'].to_numpy()[train_idx]
x = list(set(test_labels))
dic = dict(zip(x, list(range(1,len(x)+1))))
labels_test =[dic[v] for v in test_labels]
x = list(set(train_labels))
dic = dict(zip(x, list(range(1,len(x)+1))))
labels_train =[dic[v] for v in train_labels]

In [ ]:
mn10_classes = ['bathtub', 'bed', 'chair', 'desk', 'dresser', 'monitor', 'night_stand', 'sofa', 'table', 'toilet']
dir = './datasets/Manifold40/raw'
classes = sorted(os.listdir(dir))
n = 0

train_labels = []
i_train = []
cntr = 0
cntr_cls = 0
for i, cls in enumerate(classes):
    n = len(os.listdir(os.path.join(dir, cls, 'train')))
    if cls in mn10_classes:
        train_labels.extend([cntr_cls]*n)
        cntr_cls += 1
        i_train.extend(list(range(cntr, cntr+n)))
    cntr += n

test_labels = []
i_test = []
cntr = 0
cntr_cls = 0
for i, cls in enumerate(classes):
    n = len(os.listdir(os.path.join(dir, cls, 'test')))
    if cls in mn10_classes:
        test_labels.extend([cntr_cls]*n)
        cntr_cls += 1
        i_test.extend(list(range(cntr, cntr+n)))
    cntr += n

len(train_labels), len(test_labels)

z_test10 = z_test[i_test]

In [ ]:
tsne = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=30)
z = tsne.fit_transform(z_test[:])
# z_tsne = tsne.fit(z_train)

In [ ]:
from cycler import cycler
plt.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')
plt.rcParams.update({'font.size': 16})
plt.rcParams.update({'lines.linewidth':1})
plt.rcParams['legend.fancybox'] = False
plt.rcParams['legend.framealpha'] = None
plt.rcParams['legend.edgecolor'] = 'inherit'
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['mathtext.rm'] = 'serif'
plt.rcParams["font.family"] = "serif"

In [ ]:
from matplotlib import ticker 
mn10_classes = ['bathtub', 'bed', 'chair', 'desk', 'dresser', 'monitor', 'night', 'sofa', 'table', 'toilet']
plt.figure()
plt.locator_params(nbins=10)
plt.scatter(z[:,0], z[:,1], c=labels_test, cmap='twilight')
plt.grid(linestyle='dashed')
cbar = plt.colorbar()
tick_locator = ticker.MaxNLocator(nbins=15)
cbar.locator = tick_locator
cbar.update_ticks()
# cbar.ax.set_yticklabels(dic.keys()) 
# plt.savefig('plots/TSNE_Manifold40_HMN_level0.pdf')